In [1]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import h5py
from src.benchmarkers import *
from src.benchmarkersV2 import *
import time
from matplotlib import pyplot as plt
np.random.seed(0)

In [2]:
def is_leaf(name, node):
    """Check if a node is a dataset (leaf)."""
    return isinstance(node, h5py.Dataset)

def find_leaf_datasets(hdf5_file):
    """Find and return paths to all leaf datasets in an HDF5 file."""
    leaf_datasets = []

    # Traverse the HDF5 file
    hdf5_file.visititems(lambda name, node: leaf_datasets.append(name) if is_leaf(name, node) else None)
    
    return leaf_datasets


In [4]:
N = [10,100,200,300,500]
#N = [10]
iterations = 10

In [6]:
generator = Generator()

In [7]:
for item in N:
    generator.create_dataset(item,f"outputs/v2/column_wise/ds_{item}")
    generator.create_arrow_file(f"outputs/v2/column_wise/ds_{item}")
    generator.create_arrow_stream(f"outputs/v2/column_wise/ds_{item}")
    

In [4]:
arrow_file_memory = ClockColumnWise().benchmark_arrow()
arrow_stream_memory
arrow_file_no_memory
arrow_stream_no_memory
arrow_parquet 
hdf5_core
hdf5_sec2